In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
listings['host_since']

0       2016-01-26
1       2015-08-20
2       2015-08-05
3       2016-04-12
4       2013-01-08
5       2013-01-08
6       2014-05-22
7       2014-05-22
8       2014-05-22
9       2015-07-21
10      2016-04-01
11      2016-05-23
12      2015-11-20
13      2015-10-30
14      2015-08-03
15      2015-08-03
16      2015-08-03
17      2015-02-01
18      2015-08-03
19      2014-08-25
20      2012-06-11
21      2012-06-11
22      2015-09-24
23      2016-05-11
24      2012-06-11
25      2013-02-19
26      2016-06-16
27      2015-12-16
28      2016-06-25
29      2016-05-11
           ...    
6578    2016-06-29
6579    2016-06-21
6580    2014-10-11
6581    2014-10-08
6582    2011-07-25
6583    2015-10-30
6584    2014-05-29
6585    2016-07-06
6586    2016-06-14
6587    2016-06-25
6588    2016-06-12
6589    2015-12-30
6590    2012-07-19
6591    2015-10-14
6592    2011-06-24
6593    2012-10-26
6594    2016-02-20
6595    2015-05-05
6596    2014-01-17
6597    2015-07-04
6598    2016-05-25
6599    2016

In [5]:
listings['last_scraped']

0       2016-07-07
1       2016-07-07
2       2016-07-07
3       2016-07-07
4       2016-07-07
5       2016-07-07
6       2016-07-07
7       2016-07-07
8       2016-07-07
9       2016-07-07
10      2016-07-07
11      2016-07-07
12      2016-07-07
13      2016-07-07
14      2016-07-07
15      2016-07-07
16      2016-07-07
17      2016-07-07
18      2016-07-07
19      2016-07-07
20      2016-07-07
21      2016-07-07
22      2016-07-07
23      2016-07-07
24      2016-07-07
25      2016-07-07
26      2016-07-07
27      2016-07-07
28      2016-07-07
29      2016-07-07
           ...    
6578    2016-07-07
6579    2016-07-07
6580    2016-07-07
6581    2016-07-07
6582    2016-07-07
6583    2016-07-07
6584    2016-07-07
6585    2016-07-07
6586    2016-07-07
6587    2016-07-07
6588    2016-07-07
6589    2016-07-07
6590    2016-07-07
6591    2016-07-07
6592    2016-07-07
6593    2016-07-07
6594    2016-07-07
6595    2016-07-07
6596    2016-07-07
6597    2016-07-07
6598    2016-07-07
6599    2016

In [6]:
listings['host_since'].dtype, listings['last_scraped'].dtype

(dtype('O'), dtype('O'))

In [22]:
def datetime_strip(x):
    x_date = datetime.datetime.strptime(x, "%Y-%m-%d")
    return x_date

In [25]:
def get_days(b):
    return b.days

In [27]:
listings['days_since_host'] = (listings['last_scraped'].map(datetime_strip) - listings['host_since'].map(datetime_strip)).map(get_days)

In [28]:
listings['days_since_host']

0        163
1        322
2        337
3         86
4       1276
5       1276
6        777
7        777
8        777
9        352
10        97
11        45
12       230
13       251
14       339
15       339
16       339
17       522
18       339
19       682
20      1487
21      1487
22       287
23        57
24      1487
25      1234
26        21
27       204
28        12
29        57
        ... 
6578       8
6579      16
6580     635
6581     638
6582    1809
6583     251
6584     770
6585       1
6586      23
6587      12
6588      25
6589     190
6590    1449
6591     267
6592    1840
6593    1350
6594     138
6595     429
6596     902
6597     369
6598      43
6599      32
6600     519
6601     586
6602     190
6603     626
6604     195
6605     672
6606    1667
6607      28
Name: days_since_host, dtype: int64